This notebooks finetunes VGG16 by adding a couple of Dense layers and trains it to classify between cats and dogs and trains it for multiple epochs

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import tensorflow as tf

from tensorflow.contrib.keras import layers
from tensorflow.contrib.keras import models
from tensorflow.contrib.keras import optimizers
from tensorflow.contrib.keras import applications
from tensorflow.contrib.keras.python.keras.preprocessing import image
from tensorflow.contrib.keras.python.keras.applications import imagenet_utils

In [3]:
def get_batches(dirpath, gen=image.ImageDataGenerator(), shuffle=True, batch_size=64, class_mode='categorical'):
    return gen.flow_from_directory(dirpath, target_size=(224,224), class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [4]:
batch_size = 64

In [5]:
train_batches = get_batches('./data/train', batch_size=batch_size)

Found 22797 images belonging to 2 classes.


In [6]:
val_batches = get_batches('./data/valid', batch_size=batch_size)

Found 2203 images belonging to 2 classes.


Model creation

In [7]:
vgg16 = applications.VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

In [8]:
##

In [9]:
finetune_in = vgg16.output

In [10]:
x = layers.Flatten(name='flatten')(finetune_in)
x = layers.Dense(4096, activation='relu', name='fc1')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(4096, activation='relu', name='fc2')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

In [11]:
predictions = layers.Dense(train_batches.num_class, activation='softmax', name='predictions')(x)

In [12]:
model = models.Model(inputs=vgg16.input, outputs=predictions)

In [13]:
##

We tell the model to train on the last 3 layers

In [14]:
for layer in model.layers[:-7]:
    layer.trainable = False

In [16]:
for i, layer in enumerate(model.layers):
   print(i, layer.name, layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False
19 flatten False
20 fc1 True
21 batch_normalization_1 True
22 dropout_1 True
23 fc2 True
24 batch_normalization_2 True
25 dropout_2 True
26 predictions True


In [17]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
epochs = 4

In [20]:
steps_per_epoch = train_batches.samples // train_batches.batch_size
validation_steps = val_batches.samples // val_batches.batch_size

In [ ]:
for i in range(epochs):
    model.fit_generator(train_batches, validation_data=val_batches, epochs=1,
                        steps_per_epoch=steps_per_epoch,validation_steps=validation_steps)
    model.save_weights('finetune%i.h5' % i)

This give us a validation score of around: `val_loss: 0.2086 - val_acc: 0.9612`

## Gen submission file

In [23]:
import submission

In [24]:
test_batches, steps = submission.test_batches()

Found 12500 images belonging to 1 classes.


In [25]:
preds = model.predict_generator(test_batches, steps)

In [26]:
preds.shape

(12500, 2)

In [28]:
submission.gen_file(preds, test_batches)

This gave a score of around `0.29` on the public leaderboard

In [29]:
submission.gen_file(preds, test_batches, name='sub-clip.csv', clip=True)

With the clip of the probabilities it gave a score of around `0.13139` on the public learderboard